In [21]:
import pathlib
import pickle
from PIL import Image
import io
from uuid import uuid4

from azure.storage.blob import ContainerClient
import matplotlib.pyplot as plt
import numpy as np
from joblib import Parallel, delayed

from experiment_utils.secrets import Secrets
from experiment_utils.utils import getLogger
from experiment_utils.constants import *

from csgo_clips_autotrim.feature_extraction import DownsampleConfig, get_downsampled_frames

In [22]:
def get_png_image_data(img_arr) -> :
    data = io.BytesIO()
    
    
    return data.getvalue()

In [33]:
def upload_task(file_path: pathlib.Path, output_folder: pathlib.Path, output_prefix: str):
    '''
    Upload frames from a given video to blob store.
    '''
    secrets = Secrets.load()
    container_client = ContainerClient.from_container_url(container_url=secrets.AZURE_BLOBSTORE_CONTAINER_URL, credential=secrets.AZURE_BLOBSTORE_SAS_TOKEN)
    log = getLogger('upload_task')
    log.info('Uploading training data from video: %s', file_path)
    video_file_name, extension = file_path.name.split('.mp4')
    frame_name_prefix = f'{video_file_name}'
    frames = get_downsampled_frames(file_path.absolute().as_posix(), DownsampleConfig.from_str("downsample_1920x1080_60_RGB"))
    for idx, frame in enumerate(frames):
        frame_file_name = f'{frame_name_prefix}_{idx:06}.png'
        # container_client.upload_blob(f'{output_prefix}/{frame_file_name}', data, overwrite=True)
        plt.imsave(output_folder / frame_file_name, frame, cmap='gray')

In [34]:
def upload_training_images(input_folder: pathlib.Path, output_folder: pathlib.Path, output_prefix: str):
    '''
    Upload frames from the clips in the input folder to blob storage.
    '''
    tasks = []
    
    for path in input_folder.glob("*.mp4"):
        if not path.is_file():
            continue
        
        tasks.append(delayed(upload_task)(path, output_folder, output_prefix))
    
    tic = time.perf_counter()
    Parallel(n_jobs=4)(tasks)
    toc = time.perf_counter()
    
    logger.info("Upload took: %dms", (toc - tic) * 1000)

In [35]:
upload_training_images(DATA_DIR / 'clips' / 'input', DATA_DIR / 'clips' / 'output', 'label-studio/src/frames')

[INFO] 2022-10-23 15:54:44,791 upload_task : Uploading training data from video: /code/data/clips/input/blue_and_ash_fail.mp4
[INFO] 2022-10-23 15:54:44,808 upload_task : Uploading training data from video: /code/data/clips/input/ashtm_tactical_nade.mp4
  0%|          | 0/3600 [00:00<?, ?it/s][INFO] 2022-10-23 15:54:44,885 upload_task : Uploading training data from video: /code/data/clips/input/ashtm_kit_useless.mp4
[INFO] 2022-10-23 15:54:44,898 upload_task : Uploading training data from video: /code/data/clips/input/B_rush_stopped.mp4
3600it [00:25, 139.90it/s] [00:25<00:00, 139.48it/s]
 17%|█▋        | 600/3600 [00:03<00:19, 150.65it/s]
3600it [00:24, 146.86it/s]                          
100%|██████████| 3600/3600 [00:20<00:00, 176.93it/s][INFO] 2022-10-23 15:57:21,446 __main__    : Upload took: 158000ms




